In [ ]:
import time
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from sneks_ai import SnakeGame, Snake, Apple  # use the classes from sneks-ai.py

In [ ]:
gameSize = 20
numSnakes = 2
numApples = 2
myFigsize = 5
game = SnakeGame(
    canvas_size = gameSize, 
    nApples = numApples, 
    nSnakes = numApples,
    annotate = False,
    keepGameLog = True
    )
snakeIDs = [snake for snake in game.snakes]
plt.figure(figsize=(myFigsize,myFigsize))
plt.imshow(game.get_state(snakeID=snakeIDs[0]))
plt.xticks([])
plt.yticks([])
plt.show()

# Outside of the game you request actions of each snake, and then update the game with the action_vector, where the action vector length is the number of snakes

In [ ]:
# Outside of the game you request actions of each snake, and then update the game with the action_vector, where the action vector length is the number of snakes
# E.G.
maxticker = 10
while not game.done:
    
    # initialize the action vector
    action_vector = [None]*numSnakes
    for snake in game.snakes:
        if game.snakes[snake].alive:
            action_vector[snake] = game.snakes[snake].take_action(game.get_state(snakeID=snake), game.choices)
    # update the game
    agnostic_state, movement_rewards, done_vector =  game.updateGame(action_vector)
    
    
totalScores = game.get_totalScores()
print(totalScores)    
if game.ticker > maxticker:
    maxticker = game.ticker
    LongestGame = game.gameLog
    # print(movement_rewards)
    # print(done_vector)
    # plt.imshow(agnostic_state)
    # # plt.imshow(game.get_state(snakeID=snake))
    # plt.xticks([])
    # plt.yticks([])
    # plt.show()
    # time.sleep(1)
    # clear_output(wait=True)
    



In [ ]:
game.snakes[snake].body

In [ ]:
alivesnakes = [snake for snake in game.snakes if game.snakes[snake].alive]
actions = [action_vector[snake] for snake in game.snakes if game.snakes[snake].alive]

# everyone takes an action based on the action vector
possiblenewheads = [[game.snakes[snake].head[0] + game.directions[game.choices[actions[i]]][0], game.snakes[snake].head[1] + game.directions[game.choices[actions[i]]][1]] for i, snake in enumerate(alivesnakes)]
possiblenewheads

In [ ]:
action_vector

In [ ]:
snakeIDS = 1
game.snakes[snakeIDS].head

In [ ]:
game.snakes[snakeIDS].body

In [ ]:
# state = game.get_state(snakeID=1)
state = game.get_agnosticState()
snakeIDS = 1
plt.imshow(state)
game.snakes[snakeIDS].head[0]

In [ ]:
action_vector

In [ ]:
action_vector = [None]*numSnakes
for snake in game.snakes:
    if game.snakes[snake].alive:
        action_vector[snake] = game.snakes[snake].take_action(game.get_state(snakeID=snake), game.choices)
        
action_vector

for apple in game.apples:
    game.apples[apple].update_value()
    # if the apple value is less that 25% of the original value, place a new apple
    if game.apples[apple].value < 0.25*game.apple_Reward:
        new_apple_pos = game.place_apple()
        game.apples[apple] = Apple(new_apple_pos, game.apple_Reward, game.gamma)
        
        
        

In [ ]:
# Update the ticker
game.ticker += 1

for snake in game.snakes:
    game.snakes[snake].scoreUpdate = 0
alivesnakes = [snake for snake in game.snakes if game.snakes[snake].alive]
alivesnakes

actions = [action_vector[snake] for snake in game.snakes if game.snakes[snake].alive]
actions

# everyone takes an action based on the action vector
possiblenewheads = [[game.snakes[snake].head[0] + game.directions[game.choices[actions[i]]][0],\
    game.snakes[snake].head[1] + game.directions[game.choices[actions[i]]][1]]\
        for i, snake in enumerate(alivesnakes)]
possiblenewheads


In [ ]:
game.directions[game.choices[actions[snakeIDS]]][1]

In [ ]:
possiblenewheads[snakeIDS]

In [ ]:
game.snakes[snakeIDS].head

In [ ]:
game.snakes[snakeIDS].body

In [ ]:
def check_hitSelf(possiblenewhead, 
                    snakeID):
    # check for given snake if he hit own body, then he is dead
    if possiblenewhead in game.snakes[snakeID].body:
        print(f"Self hit, {snakeID} died!")
def check_hitWall(possiblenewhead, 
                      snakeID):
        if possiblenewhead[0] < 0 or possiblenewhead[0] >= game.canvas_size or possiblenewhead[1] < 0 or possiblenewhead[1] >= game.canvas_size:
                print(f"Wall hit, {snakeID} died!")
def hunger_update(snakeID):
        game.snakes[snakeID].hunger += 1
        if game.snakes[snakeID].hunger > game.snake_dieHunger:
            print(f"Hunger too great, {snakeID} died!")
            
for i in range(len(alivesnakes)):
    game.snakes[alivesnakes[i]].alive
    check_hitSelf(possiblenewheads[i], alivesnakes[i])
    check_hitWall(possiblenewheads[i], alivesnakes[i])
    hunger_update(alivesnakes[i])

In [ ]:
alivesnakes

In [ ]:
[game.snakes[snakeIDS].head] + game.snakes[snakeIDS].body

In [ ]:
possiblenewheads[snakeIDS]

In [ ]:
[game.snakes[snakeIDS].head] + game.snakes[snakeIDS].body[:-1]

In [ ]:
[game.snakes[i].head] + game.snakes[i].body[:-1]

In [ ]:
import random
random.randint(3, game.canvas_size-4)

In [ ]:
game.ticker

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(game.get_state(snakeID=1))